In [6]:
%load_ext blackcellmagic

In [7]:
import tweepy
import re
from decouple import config
import json
import requests
import sys

TWITTER_AUTH = tweepy.OAuthHandler(config('TWITTER_CONSUMER_KEY'),config('TWITTER_CONSUMER_SECRET'))

TWITTER_AUTH.set_access_token(config('TWITTER_ACCESS_TOKEN'),config('TWITTER_ACCESS_TOKEN_SECRET'))

TWITTER = tweepy.API(TWITTER_AUTH)

In [8]:
TWITTER_ACCESS_TOKEN =config('TWITTER_ACCESS_TOKEN')
TWITTER_ACCESS_TOKEN_SECRET =config('TWITTER_ACCESS_TOKEN_SECRET')

# 32 tweets per bert request

In [17]:
import tweepy
import re
from decouple import config
import json
import requests
import sys


def bert_request(timeline):
    tweet_list = [tweet["tweet"]["tweet"] for tweet in timeline]
    data = {"description": tweet_list, "max_seq_length": 32}
    headers = {"Content-type": "application/json", "cache-control": "no-cache"}
    data = json.dumps(data)
    results = requests.post("http://35.222.5.199:5000/", data=data, headers=headers)
    # results is a list comprehension of zipping tweet & bert_result lists 
    # and making two dictionary key,values out of each.
    output = {
        "results": [
            {"tweet": t, "bert_result": r}
            for t, r in zip(timeline, results.json()["results"])
        ]
    }
    return output


def process_tweet(full_tweet):
    tweet = full_tweet.full_text
    # strip username
    tweet = re.sub(r"(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)", "", tweet)
    # strip newlines and unicode characters that aren't formatted
    tweet = re.sub(r"\n|&gt;|RT :", "", tweet)
    # strip twitter urls from tweets
    tweet = re.sub(
        r"(?<=^|(?<=[^a-zA-Z0-9-_\.]))https://t.co/([A-Za-z0-9-_,\']+)", "", tweet
    )
    # Remove emojis
    RE_EMOJI = re.compile("[\U00010000-\U0010ffff]", flags=re.UNICODE)
    tweet = RE_EMOJI.sub(r"", tweet)
    # remove whitespace
    tweet = tweet.strip()
    # make api request for toxicity analysis

    tweet_info = {
        "tweet": {
            "user_id": full_tweet.user.id,
            "user_name": full_tweet.user.name,
            "tweet": full_tweet.full_text,
            "tweet_id": full_tweet.id_str,
        }
    }
    return tweet_info


def clean_timeline(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET):
    # Create Twitter Connection
    twitter_auth = tweepy.OAuthHandler(
        config("TWITTER_CONSUMER_KEY"), config("TWITTER_CONSUMER_SECRET")
    )
    access_token = TWITTER_ACCESS_TOKEN
    access_token_secret = TWITTER_ACCESS_TOKEN_SECRET
    twitter_auth.set_access_token(access_token, access_token_secret)
    global TWITTER
    TWITTER = tweepy.API(twitter_auth)

    try:
        home_timeline = TWITTER.home_timeline(
            count=32,
            tweet_mode="extended",
            exlude_rts=False,
            exclude_replies=False
        )
        timeline = [process_tweet(full_tweet) for full_tweet in home_timeline]
        output = bert_request(timeline)
        return json.dumps(output)

    except tweepy.TweepError:
        print("tweepy.TweepError")

    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)


def process_request(request):
    """ Responds to a GET request with "Hello world!". Forbids a PUT request.
    Args:
        request (flask.Request): The request object.
        <http://flask.pocoo.org/docs/1.0/api/#flask.Request>
    Returns:
        The response text, or any set of values that can be turned into a
         Response object using `make_response`
        <http://flask.pocoo.org/docs/1.0/api/#flask.Flask.make_response>.
    """
    from flask import abort

    content_type = request.headers["content-type"]
    request_json = request.get_json(silent=True)
    request_args = request.args

    if content_type == "application/json":
        request_json = request.get_json(silent=True)
        # TWITTER_ACCESS_TOKEN check/set/error
        if request_json and "TWITTER_ACCESS_TOKEN" in request_json:
            TWITTER_ACCESS_TOKEN = request_json["TWITTER_ACCESS_TOKEN"]
        else:
            raise ValueError("Missing a 'TWITTER_ACCESS_TOKEN'")
        # TWITTER_ACCESS_TOKEN_SECRET check/set/error
        if request_json and "TWITTER_ACCESS_TOKEN_SECRET" in request_json:
            TWITTER_ACCESS_TOKEN_SECRET = request_json["TWITTER_ACCESS_TOKEN_SECRET"]
        else:
            raise ValueError("Missing a 'TWITTER_ACCESS_TOKEN_SECRET'")

        if request_json and "since_id" in request_json:
            since_id = request_json["since_id"]
        else:
            since_id = None

        # Call the function for the POST request.
        if request.method == "POST":
            return clean_timeline(
                TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET, since_id
            )
    else:
        return abort(405)

In [18]:
r = clean_timeline(TWITTER_ACCESS_TOKEN,TWITTER_ACCESS_TOKEN_SECRET)

In [20]:
r =json.loads(r)

In [23]:
len(r["results"])

31

In [27]:
home_timeline = TWITTER.home_timeline(
    count=32, tweet_mode="extended", exlude_rts=False, exclude_replies=False
)
timeline = [process_tweet(full_tweet) for full_tweet in home_timeline]

In [29]:
len(timeline)

31

In [10]:
headers = {"Content-Type":"application/json"}
body = {"TWITTER_ACCESS_TOKEN": config('TWITTER_ACCESS_TOKEN'),
        "TWITTER_ACCESS_TOKEN_SECRET": config('TWITTER_ACCESS_TOKEN_SECRET')
        }

In [11]:
%%time
r = requests.post("https://us-central1-twitter-bert-models.cloudfunctions.net/function-1",
                  headers=headers,
                  data=json.dumps(body))

Wall time: 520 ms


In [14]:
r.text

'OK'

# single tweet per bert request

In [11]:
import tweepy
import re
from decouple import config
import json
import requests
import sys


def bert_request(tweet_text):
    data = {"description": tweet_text}
    headers = {"Content-type": "application/json", "cache-control": "no-cache"}
    data = json.dumps(data)
    return requests.post("http://35.192.71.216:5000/", data=data, headers=headers)


def process_tweet(full_tweet):
    # strip username
    tweet = full_tweet.full_text

    tweet = re.sub(r"(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)", "", tweet)
    # strip newlines and unicode characters that aren't formatted
    tweet = re.sub(r"\n|&gt;|RT :", "", tweet)
    # strip twitter urls from tweets
    tweet = re.sub(
        r"(?<=^|(?<=[^a-zA-Z0-9-_\.]))https://t.co/([A-Za-z0-9-_,\']+)", "", tweet
    )
    # Remove emojis
    RE_EMOJI = re.compile("[\U00010000-\U0010ffff]", flags=re.UNICODE)
    tweet = RE_EMOJI.sub(r"", tweet)
    # remove whitespace
    tweet = tweet.strip()
    # make api request for toxicity analysis
    bert_result = bert_request(tweet).json()

    tweet_info = {
        "tweet": {
            "user_id": full_tweet.user.id,
            "user_name": full_tweet.user.name,
            "tweet": full_tweet.full_text,
            "tweet_id": full_tweet.id_str,
            "bert_output": bert_result,
            "tweet": tweet,
        }
    }
    return tweet_info


def clean_timeline(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET):
    # Create Twitter Connection
    twitter_auth = tweepy.OAuthHandler(
        config("TWITTER_CONSUMER_KEY"), config("TWITTER_CONSUMER_SECRET")
    )
    access_token = TWITTER_ACCESS_TOKEN
    access_token_secret = TWITTER_ACCESS_TOKEN_SECRET
    twitter_auth.set_access_token(access_token, access_token_secret)
    global TWITTER
    TWITTER = tweepy.API(twitter_auth)

    try:
        home_timeline = TWITTER.home_timeline(
            count=20, tweet_mode="extended", exlude_rts=False
        )
        timeline = [process_tweet(full_tweet) for full_tweet in home_timeline]
        return json.dumps(timeline)

    except tweepy.TweepError:
        print("tweepy.TweepError")

    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)


def process_request(request):
    """ Responds to a GET request with "Hello world!". Forbids a PUT request.
    Args:
        request (flask.Request): The request object.
        <http://flask.pocoo.org/docs/1.0/api/#flask.Request>
    Returns:
        The response text, or any set of values that can be turned into a
         Response object using `make_response`
        <http://flask.pocoo.org/docs/1.0/api/#flask.Flask.make_response>.
    """
    from flask import abort

    content_type = request.headers["content-type"]
    request_json = request.get_json(silent=True)
    request_args = request.args

    if content_type == "application/json":
        request_json = request.get_json(silent=True)
        # TWITTER_ACCESS_TOKEN check/set/error
        if request_json and "TWITTER_ACCESS_TOKEN" in request_json:
            TWITTER_ACCESS_TOKEN = request_json["TWITTER_ACCESS_TOKEN"]
        else:
            raise ValueError("Missing a 'TWITTER_ACCESS_TOKEN'")
        # TWITTER_ACCESS_TOKEN_SECRET check/set/error
        if request_json and "TWITTER_ACCESS_TOKEN_SECRET" in request_json:
            TWITTER_ACCESS_TOKEN_SECRET = request_json["TWITTER_ACCESS_TOKEN_SECRET"]
        else:
            raise ValueError("Missing a 'TWITTER_ACCESS_TOKEN_SECRET'")

        # Call the function for the POST request.
        if request.method == "POST":
            return clean_timeline(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)
    else:
        return abort(405)

## TEST - Single

In [12]:
headers = {"Content-Type":"application/json"}
body = {"TWITTER_ACCESS_TOKEN": config('TWITTER_ACCESS_TOKEN'),
        "TWITTER_ACCESS_TOKEN_SECRET": config('TWITTER_ACCESS_TOKEN_SECRET')
        }

In [13]:
%%time
r = requests.post("https://us-central1-twitter-bert-models.cloudfunctions.net/function-1",
                  headers=headers,
                  data=json.dumps(body))

Wall time: 6.53 s


In [14]:
r2 = r.json()

In [15]:
r2[0]

{'tweet': {'tweet': {'user_id': 97530284,
   'user_name': 'Ben Carlson',
   'tweet': "Preview of tomorrow's Animal Spirits where Michael vows to get in shape https://t.co/udX2FrIO5e",
   'tweet_id': '1128355489253924864'}},
 'bert_result': {'identity_hate': 4.118680953979492e-05,
  'insult': 0.00019314885139465332,
  'obscene': 0.00017538666725158691,
  'severe_toxic': 3.787875175476074e-05,
  'threat': 2.753734588623047e-05,
  'toxic': 0.000570446252822876}}

In [16]:
[r2[i]["tweet"] for i in range(len(r2))]

[{'tweet': {'user_id': 97530284,
   'user_name': 'Ben Carlson',
   'tweet': "Preview of tomorrow's Animal Spirits where Michael vows to get in shape https://t.co/udX2FrIO5e",
   'tweet_id': '1128355489253924864'}},
 {'tweet': {'user_id': 23909137,
   'user_name': 'Larry Brown',
   'tweet': 'RT @SteveDelSports: Terry Rozier implies Kyrie Irving ignored Celtics coaches, game plan https://t.co/IH57mFJehK',
   'tweet_id': '1128355038034845696'}},
 {'tweet': {'user_id': 293479676,
   'user_name': 'Tohad',
   'tweet': "I'm going in Japan very soon. If you are in Tokyo or Kyoto, let me know in PM! I would love to have a drink with friends or new friends 🇯🇵",
   'tweet_id': '1128354909533949952'}},
 {'tweet': {'user_id': 87829185,
   'user_name': 'B. Miller',
   'tweet': 'Tim Conway was a funny dude.',
   'tweet_id': '1128354407811358722'}},
 {'tweet': {'user_id': 1122333511,
   'user_name': 'Sebastian Wiesendahl',
   'tweet': 'RT @doug_parr: Cement is responsible for 7% of global CO2 emission

# with Asyncio
did not work

In [17]:
import tweepy
import re
from decouple import config
import json
import requests
import sys

def bert_request(tweet_text):
    data = {"description":tweet_text}
    headers= {'Content-type':"application/json",
               "cache-control":"no-cache"
              }
    data= json.dumps(data)
    return requests.post("http://35.192.71.216:5000/", data = data, headers = headers)

def process_tweet(full_tweet):
    #strip username
    tweet = full_tweet.full_text

    tweet = re.sub(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)','',tweet)
    #strip newlines and unicode characters that aren't formatted
    tweet = re.sub(r'\n|&gt;|RT :','',tweet)
    #strip twitter urls from tweets
    tweet = re.sub(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))https://t.co/([A-Za-z0-9-_,\']+)','',tweet)
    #Remove emojis
    RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    tweet = RE_EMOJI.sub(r'', tweet)
    #remove whitespace
    tweet = tweet.strip()
    #make api request for toxicity analysis
    bert_result = bert_request(tweet).json()
    
    tweet_info = {"tweet": 
                     {"user_id": full_tweet.user.id,
                      "user_name" : full_tweet.user.name,
                      "tweet": full_tweet.full_text,
                      "tweet_id" : full_tweet.id_str,
                     "bert_output": bert_result,
                     "tweet":tweet}
                    }
    return tweet_info
    

def clean_timeline(TWITTER_ACCESS_TOKEN,TWITTER_ACCESS_TOKEN_SECRET):
    # Create Twitter Connection
    twitter_auth = tweepy.OAuthHandler(config('TWITTER_CONSUMER_KEY'),config('TWITTER_CONSUMER_SECRET'))
    access_token = TWITTER_ACCESS_TOKEN
    access_token_secret = TWITTER_ACCESS_TOKEN_SECRET
    twitter_auth.set_access_token(access_token, access_token_secret)
    global TWITTER
    TWITTER = tweepy.API(twitter_auth)
    
    try:
        home_timeline = TWITTER.home_timeline(count=20,
                                         tweet_mode='extended',
                                         exlude_rts=False)
        timeline = [ process_tweet(full_tweet)
                   for full_tweet in home_timeline]
        return json.dumps(timeline)

    except tweepy.TweepError:
        print("tweepy.TweepError")
        
    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)
        
        
def process_request(request):
    """ Responds to a GET request with "Hello world!". Forbids a PUT request.
    Args:
        request (flask.Request): The request object.
        <http://flask.pocoo.org/docs/1.0/api/#flask.Request>
    Returns:
        The response text, or any set of values that can be turned into a
         Response object using `make_response`
        <http://flask.pocoo.org/docs/1.0/api/#flask.Flask.make_response>.
    """
    from flask import abort

    content_type = request.headers['content-type']
    request_json = request.get_json(silent=True)
    request_args = request.args

    if content_type == 'application/json': 
        request_json = request.get_json(silent=True)
        # TWITTER_ACCESS_TOKEN check/set/error
        if request_json and 'TWITTER_ACCESS_TOKEN' in request_json:
            TWITTER_ACCESS_TOKEN = request_json['TWITTER_ACCESS_TOKEN']
        else:
            raise ValueError("Missing a 'TWITTER_ACCESS_TOKEN'")
        # TWITTER_ACCESS_TOKEN_SECRET check/set/error
        if request_json and 'TWITTER_ACCESS_TOKEN_SECRET' in request_json:
            TWITTER_ACCESS_TOKEN_SECRET = request_json['TWITTER_ACCESS_TOKEN_SECRET']
        else:
            raise ValueError("Missing a 'TWITTER_ACCESS_TOKEN_SECRET'")
        
        # Call the function for the POST request. 
        if request.method == 'POST':
            return clean_timeline(TWITTER_ACCESS_TOKEN,TWITTER_ACCESS_TOKEN_SECRET)
    else:
        return abort(405)

In [18]:
def clean_timeline(TWITTER_ACCESS_TOKEN,TWITTER_ACCESS_TOKEN_SECRET):
    # Create Twitter Connection
    twitter_auth = tweepy.OAuthHandler(config('TWITTER_CONSUMER_KEY'),config('TWITTER_CONSUMER_SECRET'))
    access_token = TWITTER_ACCESS_TOKEN
    access_token_secret = TWITTER_ACCESS_TOKEN_SECRET
    twitter_auth.set_access_token(access_token, access_token_secret)
    global TWITTER
    TWITTER = tweepy.API(twitter_auth)
    
    try:
        home_timeline = TWITTER.home_timeline(count=200,
                                         tweet_mode='extended',
                                         exlude_rts=False)
        timeline = [ process_tweet(full_tweet)
                   for full_tweet in home_timeline]
        return json.dumps(timeline)

    except tweepy.TweepError:
        print("tweepy.TweepError")
        
    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)
def process_tweet(full_tweet):
    #strip username
    tweet = full_tweet.full_text

    tweet = re.sub(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)','',tweet)
    #strip newlines and unicode characters that aren't formatted
    tweet = re.sub(r'\n|&gt;|RT :','',tweet)
    #strip twitter urls from tweets
    tweet = re.sub(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))https://t.co/([A-Za-z0-9-_,\']+)','',tweet)
    #Remove emojis
    RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    tweet = RE_EMOJI.sub(r'', tweet)
    #remove whitespace
    tweet = tweet.strip()
    return tweet
    #make api request for toxicity analysis
#     bert_result = bert_request(tweet).json()
    
#     tweet_info = {"tweet": 
#                      {"user_id": full_tweet.user.id,
#                       "user_name" : full_tweet.user.name,
#                       "tweet": full_tweet.full_text,
#                       "tweet_id" : full_tweet.id_str,
#                      "bert_output": bert_result,
#                      "tweet":tweet}
#                     }
#     return tweet_info




In [19]:
%%time
clean_timeline(TWITTER_ACCESS_TOKEN,TWITTER_ACCESS_TOKEN_SECRET)

Wall time: 786 ms


'["Preview of tomorrow\'s Animal Spirits where Michael vows to get in shape", "Terry Rozier implies Kyrie Irving ignored Celtics coaches, game plan", "I\'m going in Japan very soon. If you are in Tokyo or Kyoto, let me know in PM! I would love to have a drink with friends or new friends", "Tim Conway was a funny dude.", "Cement is responsible for 7% of global CO2 emissions and one of the hardest to decarbonise So the 4th-largest cement manuf\\u2026", "Someone just applied to  after finding out about Lambda from .", "KISSINGER ON KISSINGER, with an introduction by... Henry Kissinger.", "Good take. It\'s notable that  does embrace paperlessness on mobile, even if it can\'t yet bite that bullet on desktop.", "Dany went to weeeeird places after Jon dumped her", "This tour of Anduril has to be my favorite company tour ever. I feel like a kid in a candy store except the candy is robots a\\u2026", "Don\'t ask questions.", "Rip takes a look at spinal anatomy and how overextension in the lumbar

In [20]:
import asyncio
from concurrent.futures import ThreadPoolExecutor
import nest_asyncio
nest_asyncio.apply()

def get_timeline(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET):
    # Create Twitter Connection
    twitter_auth = tweepy.OAuthHandler(config('TWITTER_CONSUMER_KEY'),config('TWITTER_CONSUMER_SECRET'))
    access_token = TWITTER_ACCESS_TOKEN
    access_token_secret = TWITTER_ACCESS_TOKEN_SECRET
    twitter_auth.set_access_token(access_token, access_token_secret)
    global TWITTER
    TWITTER = tweepy.API(twitter_auth)
    
    try:
        home_timeline = TWITTER.home_timeline(count=200,
                                         tweet_mode='extended',
                                         exlude_rts=False)
        return home_timeline

    except tweepy.TweepError:
        print("tweepy.TweepError")
        
    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)
              

def asycnio_process_tweet(session, full_tweet, tweet_list):
    #strip username
    tweet = full_tweet.full_text

    tweet = re.sub(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9-_]+)','',tweet)
    #strip newlines and unicode characters that aren't formatted
    tweet = re.sub(r'\n|&gt;|RT :','',tweet)
    #strip twitter urls from tweets
    tweet = re.sub(r'(?<=^|(?<=[^a-zA-Z0-9-_\.]))https://t.co/([A-Za-z0-9-_,\']+)','',tweet)
    #Remove emojis
    RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)
    tweet = RE_EMOJI.sub(r'', tweet)
    #remove whitespace
    tweet = tweet.strip()
    tweet_list.append(tweet)

async def get_data_asynchronous(TWITTER_ACCESS_TOKEN,TWITTER_ACCESS_TOKEN_SECRET):
    home_timeline = get_timeline(TWITTER_ACCESS_TOKEN,TWITTER_ACCESS_TOKEN_SECRET)
    tweet_list =[] 
    with ThreadPoolExecutor(max_workers=1) as executor:
        with requests.Session() as session:
            # Set any session parameters here before calling `fetch`

            # Initialize the event loop        
            loop = asyncio.get_event_loop()

            # Use list comprehension to create a list of
            # tasks to complete.
            tasks = [ loop.run_in_executor(
                executor, asycnio_process_tweet,*(session, full_tweet, tweet_list) 
            ) for full_tweet in home_timeline]
            
            # Initializes the tasks to run and awaits their results
            for response in await asyncio.gather(*tasks):
                pass
    return tweet_list
def execute_async_event_loop(TWITTER_ACCESS_TOKEN,TWITTER_ACCESS_TOKEN_SECRET):
    """Really couldn't think of a proper name. 
    This function does something analogous to compiling the get_data_asynchronously funciont,
    Then it executre loop."""
    future = asyncio.create_task(get_data_asynchronous(TWITTER_ACCESS_TOKEN,TWITTER_ACCESS_TOKEN_SECRET))
    loop = asyncio.get_event_loop()
    loop.run_until_complete(future)

In [21]:
%%time
tweet_list = []
execute_async_event_loop(TWITTER_ACCESS_TOKEN, TWITTER_ACCESS_TOKEN_SECRET)

Wall time: 1.05 s
